# Análise de Estatística Descritiva dos Dados do Brasileirão

## 01 - Setup

In [1]:
def get_separador() -> str:
  sistema_operacional = platform.system()
  separador = ""

  if sistema_operacional == "Windows":
    separador = "\\"
  elif sistema_operacional == "Linux":
    separador = "/"
  
  return separador

# Importação das bibliotecas
import pandas as pd
import platform
import os

# Pegando o arquivo .csv do dataset
separador = get_separador()
caminho_atual = os.getcwd()
caminho_dataset = os.path.join(caminho_atual, f"..{separador}dataset{separador}brasileirao.csv")

# Este dataset foi baixado contendo os dados de jogos ocorridos até o dia 13/05/2025
brasileirao = pd.read_csv(caminho_dataset)

## 02 - Análises

### 02.1 - Desempenho médio de cada time por temporada

Crie tabelas, cujas linhas são os _times_, as colunas são as _temporadas_ e a variável em análise é a _quantidade de pontos_, para as seguintes informações:
  - Média [X]
  - Mediana [X]
  - Desvio Padrão [X]
  - Máximo []
  - Mínimo []

Destaque o ano com maior variação

Numa disputa entre dois times:
  - Em caso de vitória, o vencedor ganha 3 pontos.
  - Em caso de empate, ambos ganham 1 ponto.

### 02.2 - Funções

In [62]:
# Funções auxiliares
def verifica_presenca_time(time: str, temporada: int) -> bool:
  jogos_time_temporada = brasileirao[
    (brasileirao["Season"] == temporada) &
    (brasileirao["Home"] == time)
  ]

  return not (jogos_time_temporada.empty)

def calcula_pontos_time(time: str, temporada: int) -> int:

  if not verifica_presenca_time(time, temporada):
    return 0
  
  jogos_time_temporada = brasileirao[
    (brasileirao["Season"] == temporada) &
    ((brasileirao["Home"] == time) | (brasileirao["Away"] == time))
  ]

  vitorias_casa = jogos_time_temporada[
    (jogos_time_temporada["Home"] == time) & 
    (jogos_time_temporada["Res"] == "H")
  ]

  vitorias_visitante = jogos_time_temporada[
    (jogos_time_temporada["Away"] == time) & 
    (jogos_time_temporada["Res"] == "A")
  ]

  empates = jogos_time_temporada[(jogos_time_temporada["Res"] == "D")]

  total_pontos = ((len(vitorias_casa) + len(vitorias_visitante)) * 3) + (len(empates) * 1)

  return total_pontos

def get_times_temporada(temporada: int) -> set[str]:
  tabela_temporada = brasileirao[(brasileirao["Season"] == temporada)]
  times = set(tabela_temporada["Home"])
  return times

def get_todos_times() -> set[str]:
  temporadas = get_temporadas()
  times = set()

  for t in temporadas:
    times_temporada = get_times_temporada(t)
    times = times.union(times_temporada)
  
  return times

def get_temporadas() -> list[str]:
  temporadas = list(set(brasileirao.Season))
  return temporadas

def get_pontos_campeonato(time: str) -> list[int]:
  temporadas = sorted(get_temporadas())
  pontos = [] 
    
  for temp in temporadas:
    pontos_temp = calcula_pontos_time(time, temp)
    pontos.append(pontos_temp)
  
  return pontos

def media_campeonato(pontos_time: list[int]) -> float:
  quant_temporadas = len(pontos_time)
  tot_pontos = sum(pontos_time)  
  media_pontos = tot_pontos / quant_temporadas
  
  return media_pontos

def mediana_campeonato(pontos_time: list[int]) -> float:
  pontos = sorted(pontos_time)
  mediana = (pontos[6] + pontos[7]) / 2
  return mediana

def desvio_padrao_campeonato(pontos_time: list[int], media: float) -> float:
  variancia = 0
  for p in pontos_time:
    variancia += (p - media) ** 2
  
  variancia /= len(pontos_time)
  desvio_padrao = variancia ** 0.5
  return desvio_padrao
  

""" def get_pontos_campeonato(time: str) -> list[str]:
  temporadas = list(set(brasileirao.Season))
  times = sorted(get_todos_times())

  for t in times:
      pontos_campeonato = []
      
      for temp in temporadas:
        pontos = calcula_pontos_time(t, temp)
        pontos_campeonato.append(pontos) """


# Funções principais
def calcula_desempenho_pontos_totais() -> pd.DataFrame:
  temporadas = sorted(list(set(brasileirao.Season)))
  times = sorted(get_todos_times())
  dados = list()

  for t in times:
    pontos_campeonato = get_pontos_campeonato(t)
    dados.append([t] + pontos_campeonato)

  desempenho_pontos_totais = pd.DataFrame(data=dados, columns=[["Time"] + temporadas])
  return desempenho_pontos_totais

def calcula_desempenho_media() -> pd.DataFrame:
  temporadas = sorted(list(set(brasileirao.Season)))
  times = sorted(get_todos_times())
  dados = list()

  for t in times:
    pontos_campeonato = get_pontos_campeonato(t)
    media_time = media_campeonato(pontos_campeonato)
    dados_time = [t] + [media_time] + pontos_campeonato
    dados.append(dados_time)
  
  colunas = ["Time", "Média"] + temporadas
  desempenho = pd.DataFrame(data=dados, columns=colunas)

  return desempenho

def calcula_desempenho_mediana() -> pd.DataFrame:
  temporadas = sorted(list(set(brasileirao.Season)))
  times = sorted(get_todos_times())
  dados = list()

  for t in times:
    pontos_campeonato = get_pontos_campeonato(t)
    mediana = mediana_campeonato(pontos_campeonato)
    dados_time = [t] + [mediana] + pontos_campeonato
    dados.append(dados_time)
  
  colunas = ["Time", "Mediana"] + temporadas
  desempenho = pd.DataFrame(data=dados, columns=colunas)

  return desempenho

def calcula_desempenho_desvio_padrao() -> pd.DataFrame:
  temporadas = sorted(list(set(brasileirao.Season)))
  times = sorted(get_todos_times())
  dados = list()

  for t in times:
    pontos_campeonato = get_pontos_campeonato(t)
    media = media_campeonato(pontos_campeonato)
    desvio_padrao = desvio_padrao_campeonato(pontos_campeonato, media)
  
    dados_time = [t, media, desvio_padrao] + pontos_campeonato
    dados.append(dados_time)
  
  colunas = ["Time", "Média", "Desvio Padrão"] + temporadas
  desempenho = pd.DataFrame(data=dados, columns=colunas)

  return desempenho


### 02.3 - Desempenho por total de pontos em cada temporada

In [38]:
calcula_desempenho_pontos_totais()

,Time,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,America MG,0,0,0,0,28,0,40,0,0,53,53,24,0,0
1,Athletico-PR,0,64,54,51,57,51,57,64,53,47,58,56,42,0
2,Atletico GO,30,0,0,0,0,36,0,0,50,53,36,0,30,0
3,Atletico-MG,72,57,62,69,62,54,59,48,68,84,58,66,47,12
4,Avai,0,0,0,42,0,43,0,20,0,0,35,0,0,0
5,Bahia,47,48,37,0,0,50,48,49,44,43,0,44,53,12
6,Botafogo RJ,55,61,34,0,59,53,51,43,27,0,53,64,79,11
7,Bragantino,0,0,0,0,0,0,0,0,53,56,44,62,44,17
8,CSA,0,0,0,0,0,0,0,32,0,0,0,0,0,0
9,Ceara,0,0,0,0,0,0,44,39,52,50,37,0,0,12


### 02.4 - Desempenho com a média

In [50]:
calcula_desempenho_media()

,Time,Média,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,America MG,14.142857,0,0,0,0,28,0,40,0,0,53,53,24,0,0
1,Athletico-PR,46.714286,0,64,54,51,57,51,57,64,53,47,58,56,42,0
2,Atletico GO,16.785714,30,0,0,0,0,36,0,0,50,53,36,0,30,0
3,Atletico-MG,58.428571,72,57,62,69,62,54,59,48,68,84,58,66,47,12
4,Avai,10.000000,0,0,0,42,0,43,0,20,0,0,35,0,0,0
5,Bahia,33.928571,47,48,37,0,0,50,48,49,44,43,0,44,53,12
6,Botafogo RJ,42.142857,55,61,34,0,59,53,51,43,27,0,53,64,79,11
7,Bragantino,19.714286,0,0,0,0,0,0,0,0,53,56,44,62,44,17
8,CSA,2.285714,0,0,0,0,0,0,0,32,0,0,0,0,0,0
9,Ceara,16.714286,0,0,0,0,0,0,44,39,52,50,37,0,0,12


### 02.5 - Desempenho com a mediana

In [53]:
calcula_desempenho_mediana()

,Time,Mediana,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,America MG,0.0,0,0,0,0,28,0,40,0,0,53,53,24,0,0
1,Athletico-PR,53.5,0,64,54,51,57,51,57,64,53,47,58,56,42,0
2,Atletico GO,0.0,30,0,0,0,0,36,0,0,50,53,36,0,30,0
3,Atletico-MG,60.5,72,57,62,69,62,54,59,48,68,84,58,66,47,12
4,Avai,0.0,0,0,0,42,0,43,0,20,0,0,35,0,0,0
5,Bahia,44.0,47,48,37,0,0,50,48,49,44,43,0,44,53,12
6,Botafogo RJ,52.0,55,61,34,0,59,53,51,43,27,0,53,64,79,11
7,Bragantino,0.0,0,0,0,0,0,0,0,0,53,56,44,62,44,17
8,CSA,0.0,0,0,0,0,0,0,0,32,0,0,0,0,0,0
9,Ceara,0.0,0,0,0,0,0,0,44,39,52,50,37,0,0,12


### 02.6 - Desempenho com o desvio padrão

In [63]:
calcula_desempenho_desvio_padrao()

,Time,Média,Desvio Padrão,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,America MG,14.142857,20.314868,0,0,0,0,28,0,40,0,0,53,53,24,0,0
1,Athletico-PR,46.714286,19.890517,0,64,54,51,57,51,57,64,53,47,58,56,42,0
2,Atletico GO,16.785714,20.277287,30,0,0,0,0,36,0,0,50,53,36,0,30,0
3,Atletico-MG,58.428571,15.904691,72,57,62,69,62,54,59,48,68,84,58,66,47,12
4,Avai,10.000000,16.557260,0,0,0,42,0,43,0,20,0,0,35,0,0,0
5,Bahia,33.928571,20.080069,47,48,37,0,0,50,48,49,44,43,0,44,53,12
6,Botafogo RJ,42.142857,23.564068,55,61,34,0,59,53,51,43,27,0,53,64,79,11
7,Bragantino,19.714286,24.653103,0,0,0,0,0,0,0,0,53,56,44,62,44,17
8,CSA,2.285714,8.241260,0,0,0,0,0,0,0,32,0,0,0,0,0,0
9,Ceara,16.714286,21.150578,0,0,0,0,0,0,44,39,52,50,37,0,0,12
